In [38]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/ML/innovation_lab

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/ML/innovation_lab'
/content/drive/MyDrive/ML/innovation_lab


In [39]:
#Function to give number of hedge words and non-hedge words

import re
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

hedge_list=['perhaps', 'maybe', 'admittedly', 'admittingly','might', 'possibly', 'likely', 'probably', 'predominantly', 'presumably', 'seems', 'seemingly', 'appears', 'may', 'think', 'suggests', 'sometimes', 'often', 'around', 'roughly', 'fairly', 'usually', 'guess']

def hedge(sent):
  hword=0
  sent=sent.lower()
  sent=re.sub("[^A-Za-z0-9\s]","",sent)
  # print(sent)
  sent_split=sent.split()
  slen=len(sent_split)
  for word in sent_split:
    if word in hedge_list:
      hword+=1
  nhword=slen-hword
  return (slen,hword,nhword)

sent='it might! rain today, and we May possibly! get drenched, so carry your umbrella.'
print(sent,hedge(sent),sep='\n')

it might! rain today, and we May possibly! get drenched, so carry your umbrella.
(14, 3, 11)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
"""
DATA EXTRACTION FROM REVIEW TEXT
"""

import os
import re
import pandas as pd
import numpy as np
from typing import Sized
import json
import matplotlib.pyplot as plt
import pdb




#LISTS
src_file_fin=[]
decision_list_fin = []
para_list_fin = []
sent_count_fin = []
word_count_fin = []
word_freq_fin = []
char_freq_fin = []
vocab_len_fin = []
hedge_fin = []
nhedge_fin = []
hedge_ratio_fin = []
section_count_fin = []
aspect_count_fin = []
section_list_fin = []
aspect_list_fin = []



#TEXT FILE ITERATION
src = '/content/drive/MyDrive/ML/innovation_lab/NIPS_2018_ann' #PATH to directory containing all annotated qualitative review txt files
src_file = sorted(os.listdir(src))
print(len(src_file))

s_count_dic = {'ABS':None, 'INT':None, 'RWK':None, 'PDI':None, 'DAT':None, 'MET':None, 'EXP':None, 'RES':None, 'TNF':None, 'ANA':None, 'FWK':None, 'OAL':None, 'BIB':None, 'EXT':None}
a_count_dic = {'APR':None, 'NOV':None, 'IMP':None, 'CMP':None, 'PNF':None, 'REC':None, 'EMP':None, 'SUB':None, 'CLA':None}


for txt_file in src_file:
  # print('\n',txt_file)
  if '_annotated' not in txt_file:
    continue
  name=txt_file.split('_annotated')[0]

  file1 = open(os.path.join(src,txt_file),"r")
  s=file1.readlines()
  # print(s,len(s))

  # 14 sections, 9 aspects
  section_dic = {'ABS':None, 'INT':None, 'RWK':None, 'PDI':None, 'DAT':None, 'MET':None, 'EXP':None, 'RES':None, 'TNF':None, 'ANA':None, 'FWK':None, 'OAL':None, 'BIB':None, 'EXT':None}
  aspect_dic = {'APR':None, 'NOV':None, 'IMP':None, 'CMP':None, 'PNF':None, 'REC':None, 'EMP':None, 'SUB':None, 'CLA':None}

  A=s[0].split(']]')
  B=[]
  for sen in A:
    sen=sen+']]'
    sen=sen.replace('"','')
    sen=sen.replace('\\n',' ')
    sen=sen.strip()
    sen=sen.replace('\\u201c','"')
    sen=sen.replace('\\u201d','"')
    sen=sen.replace('\\u2018','\'')
    sen=sen.replace('\\u2019','\'')
    sen=sen.replace('\\u2026','...')
    sen=sen.replace('\\','')
    sen=sen.replace('=',' ')
    B.append(sen)
  B.pop()

  if len(B)==0:
    continue

  src_file_fin.append(name)

  sec_list=[]
  asp_list=[]
  para_list=[]
  wrd_len_list=[]
  for sen in B:
    # print(sen)
    sentence = sen.split("[[")[0]
    para_list.append(sentence)
    labels = "[["+sen.split("[[")[-1]
    labels=labels[1:-1]
    # print(type(labels), labels)
    sections=labels.split('],')[0].strip()+']'
    # print(type(sections),sections)
    aspects=labels.split('],')[1].strip()+']'

    section_list=sections.split(',')
    # print(type(section_list),section_list)
    if len(section_list)>1:
      section_list[0]=section_list[0][1:]
      section_list[-1]=section_list[-1][:-1]
    else:
      section_list[0]=section_list[0][1:-1]
    # print('1',section_list)
    aspect_list=aspects.split(',')
    if len(aspect_list)>1:
      aspect_list[0]=aspect_list[0][1:]
      aspect_list[-1]=aspect_list[-1][:-1]
    else:
      aspect_list[0]=aspect_list[0][1:-1]

    for i in range(len(section_list)):
      section_list[i] = section_list[i].split('-')
    # print('2',section_list,sec_list)
    for i in range(len(aspect_list)):
      aspect_list[i] = aspect_list[i].split('-')

    for i in section_list:
      i[0]=(i[0].upper()).strip()
      if i[0] in section_dic.keys() and i[0] not in sec_list:
        sec_list.append(i[0])
    # print('3',section_list,sec_list)
    for i in aspect_list:
      i[0]=(i[0].upper()).strip()
      if i[0] in aspect_dic.keys() and i[0] not in asp_list:
        asp_list.append(i[0])

    for item in section_list:
      item[0]=item[0].upper()
      if item[0]=='NULL':
        break
      # print(item, section_dic[item[0]], type(section_dic[item[0]]))
      if item[0] not in section_dic.keys():
        continue
      if not section_dic[item[0]]:
        section_dic[item[0]]=[sentence]
      else:
        section_dic[item[0]].append(sentence)
    for item in aspect_list:
      item[0]=item[0].upper()
      if item[0]=='NULL':
        break
      # print(item, len(aspect_dic[item[0]]))
      if item[0] not in aspect_dic.keys():
        continue
      if not aspect_dic[item[0]]:
        aspect_dic[item[0]]=[sentence]
      else:
        aspect_dic[item[0]].append(sentence)


  section_count_fin.append(len(sec_list))
  aspect_count_fin.append(len(asp_list))

  para=(' ').join(para_list)
  para_list_fin.append(para)
  sent_count_fin.append(len(B))
  wrd_count,hcount,nhcount=hedge(para)[0],hedge(para)[1],hedge(para)[2]
  word_count_fin.append(wrd_count)
  word_freq_fin.append(wrd_count/len(para.split('. ')))
  para=para.lower()
  para=re.sub("\."," ",para)
  para=re.sub("[^A-Za-z\s]","",para)
  para=re.sub("\s\S{1}\s"," ",para)
  wrd_lst=para.split()
  for i in wrd_lst:
    wrd_len_list.append(len(i.strip()))
  if len(wrd_len_list)>0:
    char_freq_fin.append(sum(wrd_len_list)/len(wrd_len_list))
  else:
    char_freq_fin.append(0)
  vocab_len_fin.append(len(set(wrd_lst)))
  hedge_fin.append(hcount)
  nhedge_fin.append(nhcount)
  if nhcount>0:
    hedge_ratio_fin.append(hcount/nhcount)
  else:
    hedge_ratio_fin.append(1)

  sec_s=[]
  asp_s=[]
  for item in section_dic:
    if section_dic[item]:
      sec_s.append(len(section_dic[item]))
    else:
      sec_s.append(0)
  # print("\n")
  for item in aspect_dic:
    if aspect_dic[item]:
      asp_s.append(len(aspect_dic[item]))
    else:
      asp_s.append(0)

  s_lst={}
  s_keys=[x for x in section_dic.keys()]
  for i in range(len(sec_s)):
    s_lst[s_keys[i]]=sec_s[i]
    # print(s_count_dic[s_keys[i]],sec_s[i])
    if not s_count_dic[s_keys[i]]:
      s_count_dic[s_keys[i]]=sec_s[i]
    else:
      s_count_dic[s_keys[i]]=s_count_dic[s_keys[i]]+sec_s[i]
  a_lst={}
  a_keys=[x for x in aspect_dic.keys()]
  for i in range(len(asp_s)):
    a_lst[a_keys[i]]=asp_s[i]
    if not a_count_dic[a_keys[i]]:
      a_count_dic[a_keys[i]]=asp_s[i]
    else:
      a_count_dic[a_keys[i]]=a_count_dic[a_keys[i]]+asp_s[i]
  section_list_fin.append(s_lst)
  aspect_list_fin.append(a_lst)

print('{} {} {} {} {} {} {} {} {} {} {} {} {} {}\n'
.format(len(src_file_fin), len(para_list_fin), len(sent_count_fin), len(word_count_fin), len(word_freq_fin), len(char_freq_fin), len(vocab_len_fin),
        len(hedge_fin), len(nhedge_fin), len(hedge_ratio_fin), len(section_count_fin), len(aspect_count_fin), len(section_list_fin), len(aspect_list_fin)))

review_dic = {'review_id':src_file_fin, 'review':para_list_fin, 'sentence count':sent_count_fin, 'word count':word_count_fin, 'avg. sentence length':word_freq_fin, 'avg. word length':char_freq_fin,
              'vocab length':vocab_len_fin, 'hedge words':hedge_fin, 'non-hedge words':nhedge_fin, 'hedge ratio':hedge_ratio_fin, 'sections covered (out of 14)':section_count_fin, 'aspects covered (out of 9)':aspect_count_fin, 
              'section distribution':section_list_fin, 'aspect distribution':aspect_list_fin}

review_df=pd.DataFrame(review_dic)
review_df

36
15 15 15 15 15 15 15 15 15 15 15 15 15 15



,review_id,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,sections covered (out of 14),aspects covered (out of 9),section distribution,aspect distribution
0,B1EA-M-0Z__R3,This paper presents a new covariance function ...,9,171,24.428571,5.165644,107,0,171,0.000000,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'..."
1,ByuP8yZRb__R2,The below review addresses the first revision ...,20,402,17.478261,5.605820,191,3,399,0.007519,6,3,"{'ABS': 0, 'INT': 0, 'RWK': 5, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 2, 'PNF'..."
2,H1meywxRW__R1,The authors of this paper propose some extensi...,7,182,22.750000,5.398876,114,1,181,0.005525,6,6,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 1, 'PNF'..."
3,NIPS_2018_184__R1,This paper presents a new type of neural netwo...,9,190,12.666667,5.227778,119,0,190,0.000000,5,2,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 0, 'PNF'..."
4,NIPS_2018_184__R2,The paper presents SplineNets that reformulate...,11,263,16.437500,5.201646,152,2,261,0.007663,6,5,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 4, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'..."
5,NIPS_2018_184__R3,"In this paper, the authors introduce SplineNet...",10,144,14.400000,5.650000,93,0,144,0.000000,10,6,"{'ABS': 0, 'INT': 1, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 1, 'PNF'..."
6,NIPS_2018_443__R1,the paper introduces a differentiable clusteri...,6,251,25.100000,5.261411,150,1,250,0.004000,7,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 0, 'PNF'..."
7,NIPS_2018_443__R2,"In this paper, the task of learning the hierar...",8,179,17.900000,5.682081,107,0,179,0.000000,8,4,"{'ABS': 0, 'INT': 3, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 2, 'CMP': 0, 'PNF'..."
8,NIPS_2018_443__R3,This paper presents a hierarchical graph repre...,9,195,21.666667,5.670213,111,1,194,0.005155,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 3, 'CMP': 0, 'PNF'..."
9,NIPS_2018_456__R1,- The problem is interesting and worth studyin...,3,22,7.333333,6.047619,18,0,22,0.000000,2,0,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'..."


In [41]:
"""
FINAL FORMULA 1:
criteria score = (no. of unique sections covered + sum(no. of sentences covering i'th section * section weight))/(total no. of sections)
+
CREATE CSV FILE
"""
#lists:
sec_score_fin = []
asp_score_fin = []
info_score_fin = []

# weights:
import json
f = open('/content/drive/MyDrive/ML/innovation_lab/golden_info/weights.json')
data = json.load(f)
s_wt_dic,a_wt_dic=list(data.values())[0],list(data.values())[1]
sec_wt=list(s_wt_dic.values())
asp_wt=list(a_wt_dic.values())
   
sec_dist_list=list(review_df['section distribution'])
asp_dist_list=list(review_df['aspect distribution'])

for count in range(review_df.shape[0]):
  sec_dist=list(sec_dist_list[count].values())[:]
  asp_dist=list(asp_dist_list[count].values())[:]
  sec_count=0
  asp_count=0
  for i in range(len(sec_dist)):
    if sec_dist[i]>0:
      sec_count+=1
      sec_dist[i]=(sec_dist[i]-1)*(sec_wt[i])
  for i in range(len(asp_dist)):
    if asp_dist[i]>0:
      asp_count+=1
      asp_dist[i]=(asp_dist[i]-1)*(asp_wt[i])

  s_len=0
  a_len=0
  for i in sec_dist:
    s_len+=i
  for i in asp_dist:
    a_len+=i

  section_score=((sec_count+s_len)/len(sec_dist))
  aspect_score=((asp_count+a_len)/len(asp_dist))
  # if section_score>=1:
  #   section_score=1
  # if aspect_score>=1:
  #   aspect_score=1
  hcount=list(review_df['hedge words'])[count]
  nhcount=list(review_df['non-hedge words'])[count]
  info_score_temp=section_score/(pow(np.e,1-aspect_score)) #intermediate informativeness score without hedge word confidence
  if nhcount>0:
    info_score=info_score_temp/(pow(np.e,(hcount/nhcount)))
  else:
    info_score=0

  sec_score_fin.append(section_score)
  asp_score_fin.append(aspect_score)
  info_score_fin.append(info_score)


scores_df=pd.DataFrame({'section score':sec_score_fin, 'aspect score':asp_score_fin, 'informativeness score':info_score_fin})
review_temp_df=pd.concat([review_df,scores_df],axis=1)
review_df.to_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/Qualitative_Review_Dataset_temp.csv',index=False)
review_temp_df

,review_id,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,sections covered (out of 14),aspects covered (out of 9),section distribution,aspect distribution,section score,aspect score,informativeness score
0,B1EA-M-0Z__R3,This paper presents a new covariance function ...,9,171,24.428571,5.165644,107,0,171,0.000000,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.571765,0.445688,0.328460
1,ByuP8yZRb__R2,The below review addresses the first revision ...,20,402,17.478261,5.605820,191,3,399,0.007519,6,3,"{'ABS': 0, 'INT': 0, 'RWK': 5, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 2, 'PNF'...",0.431786,0.337804,0.221011
2,H1meywxRW__R1,The authors of this paper propose some extensi...,7,182,22.750000,5.398876,114,1,181,0.005525,6,6,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 1, 'PNF'...",0.429203,0.667289,0.306034
3,NIPS_2018_184__R1,This paper presents a new type of neural netwo...,9,190,12.666667,5.227778,119,0,190,0.000000,5,2,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 0, 'PNF'...",0.358108,0.222844,0.164626
4,NIPS_2018_184__R2,The paper presents SplineNets that reformulate...,11,263,16.437500,5.201646,152,2,261,0.007663,6,5,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 4, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.429849,0.558231,0.274240
5,NIPS_2018_184__R3,"In this paper, the authors introduce SplineNet...",10,144,14.400000,5.650000,93,0,144,0.000000,10,6,"{'ABS': 0, 'INT': 1, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 1, 'PNF'...",0.715998,0.668533,0.513993
6,NIPS_2018_443__R1,the paper introduces a differentiable clusteri...,6,251,25.100000,5.261411,150,1,250,0.004000,7,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 0, 'PNF'...",0.500907,0.445688,0.286606
7,NIPS_2018_443__R2,"In this paper, the task of learning the hierar...",8,179,17.900000,5.682081,107,0,179,0.000000,8,4,"{'ABS': 0, 'INT': 3, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 2, 'CMP': 0, 'PNF'...",0.572435,0.444492,0.328452
8,NIPS_2018_443__R3,This paper presents a hierarchical graph repre...,9,195,21.666667,5.670213,111,1,194,0.005155,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 3, 'CMP': 0, 'PNF'...",0.572061,0.445162,0.326769
9,NIPS_2018_456__R1,- The problem is interesting and worth studyin...,3,22,7.333333,6.047619,18,0,22,0.000000,2,0,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.142976,0.000000,0.052598


In [42]:
import torch
from torch import nn

!pip install transformers
!pip install sentencepiece
from transformers import XLNetTokenizer, XLNetModel

!pip install syntok
import pandas as pd
import numpy as np
import regex as re
import syntok.segmenter as segmenter

from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
## Run
class cue_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.xlmodel = XLNetModel.from_pretrained('xlnet-base-cased')
        self.lin = nn.Linear(768,4)
        
    def forward(self,x,att):
        xl=self.xlmodel(x,attention_mask=att)[0]
        xl = xl.view(-1,xl.shape[2])
        lin = self.lin(xl)
        return (lin)

## Run
class scoperes_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.xlmodel = XLNetModel.from_pretrained('xlnet-base-cased')
        self.lin = nn.Linear(768,2)
        
    def forward(self,x,att):
        xl=self.xlmodel(x,attention_mask=att)[0]
        xl = xl.view(-1,xl.shape[2])
        lin = self.lin(xl)
        return (lin)

In [44]:
## Run
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [45]:
## Run
model = torch.load('/content/drive/MyDrive/ML/innovation_lab/Hedge cues and span detection model/cue_only_model_ep3.pt',map_location=torch.device(device))
# model.eval()

In [46]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
# s = 'I assume that it might do it.'
# encodings = tokenizer.encode_plus(s,add_special_tokens=True,return_tensors='pt',return_token_type_ids=False, return_attention_mask=True)

In [47]:
data=review_temp_df
data

,review_id,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,sections covered (out of 14),aspects covered (out of 9),section distribution,aspect distribution,section score,aspect score,informativeness score
0,B1EA-M-0Z__R3,This paper presents a new covariance function ...,9,171,24.428571,5.165644,107,0,171,0.000000,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.571765,0.445688,0.328460
1,ByuP8yZRb__R2,The below review addresses the first revision ...,20,402,17.478261,5.605820,191,3,399,0.007519,6,3,"{'ABS': 0, 'INT': 0, 'RWK': 5, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 2, 'PNF'...",0.431786,0.337804,0.221011
2,H1meywxRW__R1,The authors of this paper propose some extensi...,7,182,22.750000,5.398876,114,1,181,0.005525,6,6,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 1, 'PNF'...",0.429203,0.667289,0.306034
3,NIPS_2018_184__R1,This paper presents a new type of neural netwo...,9,190,12.666667,5.227778,119,0,190,0.000000,5,2,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 0, 'PNF'...",0.358108,0.222844,0.164626
4,NIPS_2018_184__R2,The paper presents SplineNets that reformulate...,11,263,16.437500,5.201646,152,2,261,0.007663,6,5,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 4, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.429849,0.558231,0.274240
5,NIPS_2018_184__R3,"In this paper, the authors introduce SplineNet...",10,144,14.400000,5.650000,93,0,144,0.000000,10,6,"{'ABS': 0, 'INT': 1, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 1, 'PNF'...",0.715998,0.668533,0.513993
6,NIPS_2018_443__R1,the paper introduces a differentiable clusteri...,6,251,25.100000,5.261411,150,1,250,0.004000,7,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 0, 'PNF'...",0.500907,0.445688,0.286606
7,NIPS_2018_443__R2,"In this paper, the task of learning the hierar...",8,179,17.900000,5.682081,107,0,179,0.000000,8,4,"{'ABS': 0, 'INT': 3, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 2, 'CMP': 0, 'PNF'...",0.572435,0.444492,0.328452
8,NIPS_2018_443__R3,This paper presents a hierarchical graph repre...,9,195,21.666667,5.670213,111,1,194,0.005155,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 3, 'CMP': 0, 'PNF'...",0.572061,0.445162,0.326769
9,NIPS_2018_456__R1,- The problem is interesting and worth studyin...,3,22,7.333333,6.047619,18,0,22,0.000000,2,0,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.142976,0.000000,0.052598


In [48]:
sent_outliers = ['"','Pros : 1 .','( 2017 ) .','Cons : 1 .','Sec .','Comments : 1 .','( 2016 ) .','Minor comments : 1 .','" 1 .','10 .','12 .','" SUMMARY .','( 4 ) .','Eq .','Weaknesses : 1 .','Detailed comments : 1 .','- Page 4 .','( 2015 ) .','Clarity : 1 .','- p .','11 .','13 .','( 2011 ) .','" Pros : 1 .','Pros ------ 1 .','Strengths : 1 .','Int .','EMNLP .','3b .','( 2008 ) .','In Eq .','C2 .','( 7 ) ?','MLHC 2017 .','In CVPR , 2016 .','In NIPS , 2017 .','( 2 ) .','- Page 8 .']
sent=[]
s=0

for i in range(data.shape[0]):
  # text=data.loc[i,'review'][2:].split("\', \'")[0].replace("\\\'" , "").replace("#","")
  # print('\n',i,type(data.loc[i,'review']),data.loc[i,'review'],sep='\n')
  if data.loc[i,'review'] is np.nan:
    text=''
  else:
    text=(data.loc[i,'review']).replace("\\\'" , "").replace("#","")
  text = re.sub(r'\\u201.', '', text)
  text = re.sub(r'\\n\\n', '\n', text)
  text = re.sub(r'\\n', ' ', text)
  text = re.sub(r'\\ n', ' ', text)
  text = re.sub("\'", "", text)
  text = re.sub(r"\\","", text)

  for paragraph in segmenter.process(text):
    no=0
    for sentence in paragraph:
      d={}
      sen = ' '.join([j.value for j in sentence])
      if(sen in sent_outliers):
        continue
      no+=1
      d['sentence'] = ' '.join([j.value for j in sentence])
      d['review_id'] = i+1
      d['sentence_id'] = no
      sent.append(d)

In [49]:
## Run
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
token_list=[]
for sen in tqdm(sent):
    d={}
    # inp_ids = tokenizer.encode_plus(sen['sentence'],add_special_tokens=True,return_tensors='pt',return_token_type_ids=False, return_attention_mask=True)
    inp_ids=tokenizer(sen['sentence'],add_special_tokens=True,return_tensors='pt',return_token_type_ids=False, return_attention_mask=True)
    d['input_ids'] = inp_ids['input_ids']
    d['attention_mask'] = inp_ids['attention_mask']
    d['sentence'] = sen['sentence']
    d['review_id'] = sen['review_id']
    d['sentence_id'] = sen['sentence_id']
    token_list.append(d)
# token_list = [tokenizer.encode_plus(i['sentence'],add_special_tokens=True,return_tensors='pt',return_token_type_ids=False, return_attention_mask=True) for i in sent]

100%|██████████| 164/164 [00:00<00:00, 3178.71it/s]


In [50]:
## run
cues_list=[]
max_len=100
model=model.eval()
with torch.no_grad():
    for t in tqdm(token_list):
        inp = t['input_ids'].to(device)
        att = t['attention_mask'].to(device)
        revid = t['review_id']
        sentid = t['sentence_id']
#         print(inp)
#         break
        pred = torch.max(model(inp,att),dim=1)[1]
        hedges = [tokenizer.decode(i) for i,j in zip(inp[0],pred) if (j==1 or j==2)]
        cues_list.append({'review_id':revid,'sentence_id':sentid,'pred':pred,'hedges':hedges})
#         print(pred)
#         print([tokenizer.decode(i) for i in inp[0]])
#         print(hedges)
#         print('*'*100)

100%|██████████| 164/164 [01:15<00:00,  2.18it/s]


In [51]:
## Run
scopeinp_list=[]
for i,cue_pred in tqdm(zip(token_list,cues_list)):
    pred = cue_pred['pred'].tolist()
    revid = i['review_id']
    sentid = i['sentence_id']
    raw_sent = i['sentence']
    idx=-1
    if(sum(pred)==0):
        scopeinp_list.append({'review_id':revid,'sentence_id':sentid,'raw_sentence':raw_sent,'input_ids':i['input_ids'],'attention_mask':i['attention_mask'],'hedge_cue':None})
    while(True):
        try:
            ip = i['input_ids'].view(-1).tolist()
            at = i['attention_mask'].view(-1).tolist()
            
            idx = pred.index(1,idx+1)
            hedge_cue = tokenizer.decode(ip[idx])
#             print('old ip :')
#             print(ip,'\n')
            ip.insert(idx, 1)
            at.append(1)
            ip = torch.tensor(ip).view(1,-1)
            at = torch.tensor(at).view(1,-1)
            scopeinp_list.append({'review_id':revid,'sentence_id':sentid,'raw_sentence':raw_sent,'input_ids':ip,'attention_mask':at,'hedge_cue':hedge_cue})
#             print(ip,'\n')
        except:
            break
    idx=-1
    while(True):
        try:
            ip = i['input_ids'].view(-1).tolist()
            at = i['attention_mask'].view(-1).tolist()

            idx = pred.index(2,idx+1)
            
#             pred.insert(idx, 2)
            ip.insert(idx, 2)
            at.append(1)
            ip_idx = idx+1
            
            hedge_cue = tokenizer.decode(ip[ip_idx])
#             print(tokenizer.decode(ip),'\n')
#             print(len(pred))
#             print(len(ip))
#             print(hedge_cue,'\n')
            while(idx+1<len(pred) and pred[idx+1]==2):
                hedge_cue += ' '+tokenizer.decode(ip[ip_idx+1])
                idx += 1
                ip_idx += 1
                
            ip = torch.tensor(ip).view(1,-1)
            at = torch.tensor(at).view(1,-1)
#             print(cue_pred['pred'].tolist(),'\n')
#             print(ip,'\n')
#             print(hedge_cue)
#             print(cue_pred['hedges'])
#             print('\n')
            scopeinp_list.append({'review_id':revid,'sentence_id':sentid,'raw_sentence':raw_sent,'input_ids':ip,'attention_mask':at,'hedge_cue':hedge_cue})
            
            
        except:
            break

164it [00:00, 14435.50it/s]


In [52]:
## Run
scope_model = torch.load('/content/drive/MyDrive/ML/innovation_lab/Hedge cues and span detection model/scoperes_model_ep3.pt',map_location=torch.device(device))
# scope_model.eval()

In [53]:
## Run
scope_list=[]
max_len=100
scope_model=scope_model.eval()
with torch.no_grad():
    for t in tqdm(scopeinp_list):
        inp = t['input_ids'].to(device)
        att = t['attention_mask'].to(device)
        revid = t['review_id']
        sentid = t['sentence_id']
        hedge = t['hedge_cue']
        raw_sent = t['raw_sentence']
        
        if(hedge==None):
            scope_list.append({'Review_id':revid,'Sentence_id':sentid,'Raw Sentence':raw_sent,'Hedged Sentence':tokenizer.decode(inp[0][:-2]),'Hedge':None,'Span':None})
            continue
            
        sc_pred = torch.max(scope_model(inp,att),dim=1)[1]
        sc = sc_pred.tolist()

        if(sum(sc)==0):
            scope_list.append({'Review_id':revid,'Sentence_id':sentid,'Raw Sentence':raw_sent,'Hedged Sentence':tokenizer.decode(inp[0][:-2]),'Hedge':hedge,'Span':None})
            continue
            
        inpsent = inp[0].tolist()
        scope_startid = sc.index(1)
        scope_endid = sc.index(0,scope_startid+1)
        
        inpsent = inpsent[:scope_endid]+[17,7967,17]+inpsent[scope_endid:]
        inpsent = inpsent[:scope_startid]+[17,7967,17]+inpsent[scope_startid:]
        inpsent = tokenizer.decode(inpsent[:-2])
        inpsent = inpsent.replace("#", "<span>",1)
        inpsent = inpsent.replace("#", "</span>",1)
        
        if(inpsent.find('<s>')>-1):
            inpsent = inpsent.replace("<s>", " <s> ",1)
            hedge_end_idx = inpsent.find(hedge)+len(hedge)
            inpsent = inpsent[:hedge_end_idx]+' <s> '+ inpsent[hedge_end_idx:]

        else:
            inpsent = inpsent.replace("</s>", " </s> ",1)
            hedge_end_idx = inpsent.find(hedge)+len(hedge)
            inpsent = inpsent[:hedge_end_idx]+' </s> '+ inpsent[hedge_end_idx:]
        
        scopetag_startid = inpsent.find('<span>')+len('<span>')
        scopetag_endid = inpsent.find('</span>')
        scope = inpsent[scopetag_startid:scopetag_endid]
        
        scope_list.append({'Review_id':revid,'Sentence_id':sentid,'Raw Sentence':raw_sent,'Hedged Sentence':inpsent,'Hedge':hedge,'Span':scope})

100%|██████████| 168/168 [00:18<00:00,  8.96it/s]


In [54]:
## Run
df = pd.DataFrame(scope_list)
df['Span']

0                                                   None
1                                                   None
2                                                   None
3        </s>  unsure about </s>  how much of the ker...
4                                                   None
                             ...                        
163                                                 None
164                                                 None
165                                                 None
166                                                 None
167                                                 None
Name: Span, Length: 168, dtype: object

In [55]:
df

,Review_id,Sentence_id,Raw Sentence,Hedged Sentence,Hedge,Span
0,1,1,This paper presents a new covariance function ...,This paper presents a new covariance function ...,None,None
1,1,2,"As a result , exact Bayesian inference with a ...","As a result, exact Bayesian inference with a d...",None,None
2,1,3,Pros : The result highlights an interesting re...,Pros : The result highlights an interesting re...,None,None
3,1,4,( Although I am unsure about how much of the k...,( Although I am <span> </s> unsure about </s...,unsure about,</s> unsure about </s> how much of the ker...
4,1,5,The paper is clear and very well written .,The paper is clear and very well written.,None,None
...,...,...,...,...,...,...
163,15,2,Its a useful datastructure .,Its a useful datastructure.,None,None
164,15,3,Not quite machine learning ... but useful for ...,Not quite machine learning... but useful for s...,None,None
165,15,4,Wish it was trained on a fast and competitive ...,Wish it was trained on a fast and competitive ...,None,None
166,15,5,Ideally you can use : https://github.com/sales...,Ideally you can use : https://github.com/sales...,None,None


In [56]:
count=0
for i in range(df.shape[0]):
  if(df.loc[i,'Hedge']==None):
    count+=1
count

128

In [57]:
df.dropna(subset=['Hedge'],inplace=True)
df

,Review_id,Sentence_id,Raw Sentence,Hedged Sentence,Hedge,Span
3,1,4,( Although I am unsure about how much of the k...,( Although I am <span> </s> unsure about </s...,unsure about,</s> unsure about </s> how much of the ker...
8,1,9,I suggest using the same axis limits for all s...,I <span> <s> suggest <s> using the same axi...,suggest,<s> suggest <s> using the same axis limits...
15,2,7,The authors suggest the use of multiple advers...,The authors <span> <s> suggest <s> the use ...,suggest,<s> suggest <s> the use of multiple advers...
19,2,11,3 may need to be chosen differently depending ...,3 <span> <s> may <s> need to be chosen diff...,may,<s> may <s> need to be chosen differently ...
20,2,12,"Without tuning lambda for each method , the em...","Without tuning lambda for each method, <span> ...",seem,the empirical experiments <s> seem <s> unf...
21,2,13,"This may also explain why , for example , the ...","This <span> <s> may <s> also explain why, f...",may,"<s> may <s> also explain why, for example,..."
25,2,17,"Basically , instead of a single adversarial mo...","Basically, instead of a single adversarial mod...",propose,<s> propose <s> to train multiple adversar...
27,2,19,"Though , there is no theoretical justification...","Though, there is <span> </s> no theoretical ...",no theoretical,</s> no theoretical </s> justification why...
28,2,20,Tuning the architecture of the single multi la...,Tuning the architecture of the single multi la...,might,<s> might <s> be as good
31,2,23,Given that there is also no theoretical argume...,Given that there is also no theoretical argume...,recommend,<s> recommend <s> to reject the paper


In [58]:
df.to_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/Qual_review_hedge_sent.csv',index=False)

In [59]:
data['hedge_score']=0

In [60]:
for i in tqdm(range(data.shape[0])):
  new_data=df[df['Review_id']==i+1].copy(deep=True).reset_index(drop=True)
  # print(new_data.shape)
  # new_data.drop(subset=['Hedge'],inplace=True)
  if data.loc[i,'review'] is np.nan:
    review=''
  else:
    review=data.loc[i,'review'].replace("\\\'" , "").replace("#","")
  # print(review)
  text=""
  for j in range(new_data.shape[0]):
    text+=" "+new_data.loc[j,'Hedge']
  hedge_score=len(text.split(" "))/len(review.split(" "))
  data.loc[i,'hedge_score']=hedge_score
  # print(hedge_score)
  # if(i==3):
  #   break

100%|██████████| 15/15 [00:00<00:00, 788.60it/s]


In [61]:
data

,review_id,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,sections covered (out of 14),aspects covered (out of 9),section distribution,aspect distribution,section score,aspect score,informativeness score,hedge_score
0,B1EA-M-0Z__R3,This paper presents a new covariance function ...,9,171,24.428571,5.165644,107,0,171,0.000000,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.571765,0.445688,0.328460,0.023121
1,ByuP8yZRb__R2,The below review addresses the first revision ...,20,402,17.478261,5.605820,191,3,399,0.007519,6,3,"{'ABS': 0, 'INT': 0, 'RWK': 5, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 2, 'PNF'...",0.431786,0.337804,0.221011,0.032178
2,H1meywxRW__R1,The authors of this paper propose some extensi...,7,182,22.750000,5.398876,114,1,181,0.005525,6,6,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 1, 'PNF'...",0.429203,0.667289,0.306034,0.027174
3,NIPS_2018_184__R1,This paper presents a new type of neural netwo...,9,190,12.666667,5.227778,119,0,190,0.000000,5,2,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 0, 'PNF'...",0.358108,0.222844,0.164626,0.031250
4,NIPS_2018_184__R2,The paper presents SplineNets that reformulate...,11,263,16.437500,5.201646,152,2,261,0.007663,6,5,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 4, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.429849,0.558231,0.274240,0.018519
5,NIPS_2018_184__R3,"In this paper, the authors introduce SplineNet...",10,144,14.400000,5.650000,93,0,144,0.000000,10,6,"{'ABS': 0, 'INT': 1, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 1, 'PNF'...",0.715998,0.668533,0.513993,0.017143
6,NIPS_2018_443__R1,the paper introduces a differentiable clusteri...,6,251,25.100000,5.261411,150,1,250,0.004000,7,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 0, 'PNF'...",0.500907,0.445688,0.286606,0.019608
7,NIPS_2018_443__R2,"In this paper, the task of learning the hierar...",8,179,17.900000,5.682081,107,0,179,0.000000,8,4,"{'ABS': 0, 'INT': 3, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 2, 'CMP': 0, 'PNF'...",0.572435,0.444492,0.328452,0.010929
8,NIPS_2018_443__R3,This paper presents a hierarchical graph repre...,9,195,21.666667,5.670213,111,1,194,0.005155,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 3, 'CMP': 0, 'PNF'...",0.572061,0.445162,0.326769,0.014851
9,NIPS_2018_456__R1,- The problem is interesting and worth studyin...,3,22,7.333333,6.047619,18,0,22,0.000000,2,0,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.142976,0.000000,0.052598,0.041667


In [62]:
data.to_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/Qual_review_hedge_score.csv',index=False)

In [63]:
"""
DATA EXTRACTION FROM REVIEW TEXT
"""

import os
import re
import pandas as pd
import numpy as np
from typing import Sized
import json
import matplotlib.pyplot as plt
import pdb

review_df=data
review_df

,review_id,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,sections covered (out of 14),aspects covered (out of 9),section distribution,aspect distribution,section score,aspect score,informativeness score,hedge_score
0,B1EA-M-0Z__R3,This paper presents a new covariance function ...,9,171,24.428571,5.165644,107,0,171,0.000000,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.571765,0.445688,0.328460,0.023121
1,ByuP8yZRb__R2,The below review addresses the first revision ...,20,402,17.478261,5.605820,191,3,399,0.007519,6,3,"{'ABS': 0, 'INT': 0, 'RWK': 5, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 2, 'PNF'...",0.431786,0.337804,0.221011,0.032178
2,H1meywxRW__R1,The authors of this paper propose some extensi...,7,182,22.750000,5.398876,114,1,181,0.005525,6,6,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 1, 'PNF'...",0.429203,0.667289,0.306034,0.027174
3,NIPS_2018_184__R1,This paper presents a new type of neural netwo...,9,190,12.666667,5.227778,119,0,190,0.000000,5,2,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 0, 'PNF'...",0.358108,0.222844,0.164626,0.031250
4,NIPS_2018_184__R2,The paper presents SplineNets that reformulate...,11,263,16.437500,5.201646,152,2,261,0.007663,6,5,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 4, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.429849,0.558231,0.274240,0.018519
5,NIPS_2018_184__R3,"In this paper, the authors introduce SplineNet...",10,144,14.400000,5.650000,93,0,144,0.000000,10,6,"{'ABS': 0, 'INT': 1, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 1, 'PNF'...",0.715998,0.668533,0.513993,0.017143
6,NIPS_2018_443__R1,the paper introduces a differentiable clusteri...,6,251,25.100000,5.261411,150,1,250,0.004000,7,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 0, 'PNF'...",0.500907,0.445688,0.286606,0.019608
7,NIPS_2018_443__R2,"In this paper, the task of learning the hierar...",8,179,17.900000,5.682081,107,0,179,0.000000,8,4,"{'ABS': 0, 'INT': 3, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 2, 'CMP': 0, 'PNF'...",0.572435,0.444492,0.328452,0.010929
8,NIPS_2018_443__R3,This paper presents a hierarchical graph repre...,9,195,21.666667,5.670213,111,1,194,0.005155,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 3, 'CMP': 0, 'PNF'...",0.572061,0.445162,0.326769,0.014851
9,NIPS_2018_456__R1,- The problem is interesting and worth studyin...,3,22,7.333333,6.047619,18,0,22,0.000000,2,0,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.142976,0.000000,0.052598,0.041667


In [64]:
#VADER SENTIMENT

!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def senti(sentence):
  sid_obj = SentimentIntensityAnalyzer()
  sentiment_dict = sid_obj.polarity_scores(sentence)
  return sentiment_dict['compound']

senti_score_fin = []

rev_text_list=list(review_df['review'])

for count in range(review_df.shape[0]):
  if rev_text_list[count] is not np.nan:
    senti_score_fin.append(senti(rev_text_list[count]))
  else:
    senti_score_fin.append(0)

review_df=pd.concat([review_df,pd.DataFrame({'vader compound sentiment':senti_score_fin})],axis=1)
review_df

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


,review_id,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,sections covered (out of 14),aspects covered (out of 9),section distribution,aspect distribution,section score,aspect score,informativeness score,hedge_score,vader compound sentiment
0,B1EA-M-0Z__R3,This paper presents a new covariance function ...,9,171,24.428571,5.165644,107,0,171,0.000000,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.571765,0.445688,0.328460,0.023121,0.8243
1,ByuP8yZRb__R2,The below review addresses the first revision ...,20,402,17.478261,5.605820,191,3,399,0.007519,6,3,"{'ABS': 0, 'INT': 0, 'RWK': 5, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 2, 'PNF'...",0.431786,0.337804,0.221011,0.032178,-0.9484
2,H1meywxRW__R1,The authors of this paper propose some extensi...,7,182,22.750000,5.398876,114,1,181,0.005525,6,6,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 1, 'PNF'...",0.429203,0.667289,0.306034,0.027174,0.9633
3,NIPS_2018_184__R1,This paper presents a new type of neural netwo...,9,190,12.666667,5.227778,119,0,190,0.000000,5,2,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 0, 'PNF'...",0.358108,0.222844,0.164626,0.031250,0.9796
4,NIPS_2018_184__R2,The paper presents SplineNets that reformulate...,11,263,16.437500,5.201646,152,2,261,0.007663,6,5,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 4, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.429849,0.558231,0.274240,0.018519,0.9871
5,NIPS_2018_184__R3,"In this paper, the authors introduce SplineNet...",10,144,14.400000,5.650000,93,0,144,0.000000,10,6,"{'ABS': 0, 'INT': 1, 'RWK': 1, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 1, 'PNF'...",0.715998,0.668533,0.513993,0.017143,0.9337
6,NIPS_2018_443__R1,the paper introduces a differentiable clusteri...,6,251,25.100000,5.261411,150,1,250,0.004000,7,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 0, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 0, 'PNF'...",0.500907,0.445688,0.286606,0.019608,0.9895
7,NIPS_2018_443__R2,"In this paper, the task of learning the hierar...",8,179,17.900000,5.682081,107,0,179,0.000000,8,4,"{'ABS': 0, 'INT': 3, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 2, 'CMP': 0, 'PNF'...",0.572435,0.444492,0.328452,0.010929,0.6790
8,NIPS_2018_443__R3,This paper presents a hierarchical graph repre...,9,195,21.666667,5.670213,111,1,194,0.005155,8,4,"{'ABS': 0, 'INT': 1, 'RWK': 0, 'PDI': 1, 'DAT'...","{'APR': 0, 'NOV': 1, 'IMP': 3, 'CMP': 0, 'PNF'...",0.572061,0.445162,0.326769,0.014851,0.9717
9,NIPS_2018_456__R1,- The problem is interesting and worth studyin...,3,22,7.333333,6.047619,18,0,22,0.000000,2,0,"{'ABS': 0, 'INT': 0, 'RWK': 0, 'PDI': 3, 'DAT'...","{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.142976,0.000000,0.052598,0.041667,0.0194


In [65]:
#POS FEATURES

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

def pos_count(text):
  noun_count=0
  adj_count=0
  verb_count=0
  adverb_count=0
  if text is np.nan:
    text=''
  tokens = nltk.word_tokenize(text.lower())
  text = nltk.Text(tokens)
  tags = nltk.pos_tag(text)
  from collections import Counter
  counts = dict(Counter(tag for word,tag in tags))
  for i in list(counts.keys()):
    if i[0]=='N':
      noun_count+=counts[i]
    if i[0]=='J':
      adj_count+=counts[i]
    if i[0]=='V':
      verb_count+=counts[i]
    if i[0]=='R':
      adverb_count+=counts[i]
  return (noun_count, adj_count, verb_count, adverb_count)

noun_count_list=[]
adj_count_list=[]
verb_count_list=[]
adverb_count_list=[]
for row in range(len(review_df['review'])):
  pos_tup=pos_count(review_df['review'][row])
  noun_count_list.append(pos_tup[0])
  adj_count_list.append(pos_tup[1])
  verb_count_list.append(pos_tup[2])
  adverb_count_list.append(pos_tup[3])
review_df['noun_count']=noun_count_list
review_df['adj_count']=adj_count_list
review_df['verb_count']=verb_count_list
review_df['adverb_count']=adverb_count_list

# print("POS counts (noun, adjective, verb, adverb): ({}, {}, {}, {})".format(review_df['noun_count'][n],review_df['adj_count'][n],review_df['verb_count'][n],review_df['adverb_count'][n]))
review_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,review_id,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,...,aspect distribution,section score,aspect score,informativeness score,hedge_score,vader compound sentiment,noun_count,adj_count,verb_count,adverb_count
0,B1EA-M-0Z__R3,This paper presents a new covariance function ...,9,171,24.428571,5.165644,107,0,171,0.000000,...,"{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.571765,0.445688,0.328460,0.023121,0.8243,50,30,21,3
1,ByuP8yZRb__R2,The below review addresses the first revision ...,20,402,17.478261,5.605820,191,3,399,0.007519,...,"{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 2, 'PNF'...",0.431786,0.337804,0.221011,0.032178,-0.9484,119,53,72,14
2,H1meywxRW__R1,The authors of this paper propose some extensi...,7,182,22.750000,5.398876,114,1,181,0.005525,...,"{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 1, 'PNF'...",0.429203,0.667289,0.306034,0.027174,0.9633,66,20,25,3
3,NIPS_2018_184__R1,This paper presents a new type of neural netwo...,9,190,12.666667,5.227778,119,0,190,0.000000,...,"{'APR': 0, 'NOV': 1, 'IMP': 0, 'CMP': 0, 'PNF'...",0.358108,0.222844,0.164626,0.031250,0.9796,58,26,28,13
4,NIPS_2018_184__R2,The paper presents SplineNets that reformulate...,11,263,16.437500,5.201646,152,2,261,0.007663,...,"{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 0, 'PNF'...",0.429849,0.558231,0.274240,0.018519,0.9871,79,37,49,4
5,NIPS_2018_184__R3,"In this paper, the authors introduce SplineNet...",10,144,14.400000,5.650000,93,0,144,0.000000,...,"{'APR': 0, 'NOV': 1, 'IMP': 1, 'CMP': 1, 'PNF'...",0.715998,0.668533,0.513993,0.017143,0.9337,43,28,23,2
6,NIPS_2018_443__R1,the paper introduces a differentiable clusteri...,6,251,25.100000,5.261411,150,1,250,0.004000,...,"{'APR': 0, 'NOV': 0, 'IMP': 1, 'CMP': 0, 'PNF'...",0.500907,0.445688,0.286606,0.019608,0.9895,72,28,43,16
7,NIPS_2018_443__R2,"In this paper, the task of learning the hierar...",8,179,17.900000,5.682081,107,0,179,0.000000,...,"{'APR': 0, 'NOV': 0, 'IMP': 2, 'CMP': 0, 'PNF'...",0.572435,0.444492,0.328452,0.010929,0.6790,51,24,34,5
8,NIPS_2018_443__R3,This paper presents a hierarchical graph repre...,9,195,21.666667,5.670213,111,1,194,0.005155,...,"{'APR': 0, 'NOV': 1, 'IMP': 3, 'CMP': 0, 'PNF'...",0.572061,0.445162,0.326769,0.014851,0.9717,58,32,35,10
9,NIPS_2018_456__R1,- The problem is interesting and worth studyin...,3,22,7.333333,6.047619,18,0,22,0.000000,...,"{'APR': 0, 'NOV': 0, 'IMP': 0, 'CMP': 0, 'PNF'...",0.142976,0.000000,0.052598,0.041667,0.0194,6,4,4,2


In [66]:
#INFO SCORE

#lists:
info_score_fin = []

# weights:
import json
f = open('/content/drive/MyDrive/ML/innovation_lab/golden_info/weights.json')
data = json.load(f)
s_wt_dic,a_wt_dic=list(data.values())[0],list(data.values())[1]
sec_wt=list(s_wt_dic.values())
asp_wt=list(a_wt_dic.values())
   
sec_dist_list=list(review_df['section distribution'])
asp_dist_list=list(review_df['aspect distribution'])

for count in range(review_df.shape[0]):
  sec_dist_dic=sec_dist_list[count]
  asp_dist_dic=asp_dist_list[count]
  sec_dist=list(sec_dist_dic.values())[:]
  asp_dist=list(asp_dist_dic.values())[:]
  sec_count=0
  asp_count=0
  for i in range(len(sec_dist)):
    if sec_dist[i]>0:
      sec_count+=1
      sec_dist[i]=(sec_dist[i]-1)*(sec_wt[i])
  for i in range(len(asp_dist)):
    if asp_dist[i]>0:
      asp_count+=1
      asp_dist[i]=(asp_dist[i]-1)*(asp_wt[i])

  s_len=0
  a_len=0
  for i in sec_dist:
    s_len+=i
  for i in asp_dist:
    a_len+=i

  section_score=((sec_count+s_len)/len(sec_dist))
  aspect_score=((asp_count+a_len)/len(asp_dist))
  # if section_score>=1:
  #   section_score=1
  # if aspect_score>=1:
  #   aspect_score=1
  nhcount=list(review_df['non-hedge words'])[count]
  hscore=list(review_df['hedge_score'])[count]

  if nhcount>0:
    info_score=section_score/((pow(np.e,1-aspect_score))*(pow(np.e,hscore)))
  else:
    info_score=0
  info_score_fin.append(info_score)

review_df=pd.concat([review_df,pd.DataFrame({'new info score':info_score_fin})],axis=1)
review_df.to_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/Qualitative_Review_Dataset_fin.csv',index=False)
review_df

,review_id,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,...,section score,aspect score,informativeness score,hedge_score,vader compound sentiment,noun_count,adj_count,verb_count,adverb_count,new info score
0,B1EA-M-0Z__R3,This paper presents a new covariance function ...,9,171,24.428571,5.165644,107,0,171,0.000000,...,0.571765,0.445688,0.328460,0.023121,0.8243,50,30,21,3,0.320953
1,ByuP8yZRb__R2,The below review addresses the first revision ...,20,402,17.478261,5.605820,191,3,399,0.007519,...,0.431786,0.337804,0.221011,0.032178,-0.9484,119,53,72,14,0.215628
2,H1meywxRW__R1,The authors of this paper propose some extensi...,7,182,22.750000,5.398876,114,1,181,0.005525,...,0.429203,0.667289,0.306034,0.027174,0.9633,66,20,25,3,0.299479
3,NIPS_2018_184__R1,This paper presents a new type of neural netwo...,9,190,12.666667,5.227778,119,0,190,0.000000,...,0.358108,0.222844,0.164626,0.031250,0.9796,58,26,28,13,0.159561
4,NIPS_2018_184__R2,The paper presents SplineNets that reformulate...,11,263,16.437500,5.201646,152,2,261,0.007663,...,0.429849,0.558231,0.274240,0.018519,0.9871,79,37,49,4,0.271279
5,NIPS_2018_184__R3,"In this paper, the authors introduce SplineNet...",10,144,14.400000,5.650000,93,0,144,0.000000,...,0.715998,0.668533,0.513993,0.017143,0.9337,43,28,23,2,0.505257
6,NIPS_2018_443__R1,the paper introduces a differentiable clusteri...,6,251,25.100000,5.261411,150,1,250,0.004000,...,0.500907,0.445688,0.286606,0.019608,0.9895,72,28,43,16,0.282167
7,NIPS_2018_443__R2,"In this paper, the task of learning the hierar...",8,179,17.900000,5.682081,107,0,179,0.000000,...,0.572435,0.444492,0.328452,0.010929,0.6790,51,24,34,5,0.324882
8,NIPS_2018_443__R3,This paper presents a hierarchical graph repre...,9,195,21.666667,5.670213,111,1,194,0.005155,...,0.572061,0.445162,0.326769,0.014851,0.9717,58,32,35,10,0.323615
9,NIPS_2018_456__R1,- The problem is interesting and worth studyin...,3,22,7.333333,6.047619,18,0,22,0.000000,...,0.142976,0.000000,0.052598,0.041667,0.0194,6,4,4,2,0.050451


In [70]:
# # meta_feat=['review_id', 'meta_decision', 'review']
# meta_feat=['review_id', 'review']


# count_feat=['sentence count', 'word count', 'avg. sentence length', 'avg. word length', 'vocab length']
# hedgelist_feat=['hedge words', 'non-hedge words','hedge ratio']
# hedgepeer_feat=['hedge_score']
# senti_feat=['vader compound sentiment']
# pos_feat=['noun_count', 'adj_count', 'verb_count', 'adverb_count']
# secasp_coverage_feat=['sections covered (out of 14)', 'aspects covered (out of 9)']

# secasp_dist_dic=['section distribution', 'aspect distribution']
# scores=['section score', 'aspect score', 'informativeness score']
# info_score=['new info score']

In [71]:
# #FEATURES MATRIX CONTAINING INDIVIDUAL COVERAGE OF EVERY SECTION AND ASPECT, FOR FUTURE WORK


# import pandas as pd

# sec_dist=list(review_df['section distribution'])
# asp_dist=list(review_df['aspect distribution'])

# ABS_list=[]
# INT_list=[]
# RWK_list=[]
# PDI_list=[]
# DAT_list=[]
# MET_list=[]
# EXP_list=[]
# RES_list=[]
# TNF_list=[]
# ANA_list=[]
# FWK_list=[]
# OAL_list=[]
# BIB_list=[]
# EXT_list=[]
# sec_list=[ABS_list, INT_list, RWK_list, PDI_list, DAT_list, MET_list, EXP_list, RES_list, TNF_list, ANA_list, FWK_list, OAL_list, BIB_list, EXT_list]

# APR_list=[]
# NOV_list=[]
# IMP_list=[]
# CMP_list=[]
# PNF_list=[]
# REC_list=[]
# EMP_list=[]
# SUB_list=[]
# CLA_list=[]
# asp_list=[APR_list, NOV_list, IMP_list, CMP_list, PNF_list, REC_list, EMP_list, SUB_list, CLA_list]

# for i in sec_dist:
#   # i=i.replace("\'", "\"")
#   # i=json.loads(i)
#   sec_dic=list(i.values())
#   for j in range(len(sec_list)):
#     sec_list[j].append(sec_dic[j])

# for i in asp_dist:
#   # i=i.replace("\'", "\"")
#   # i=json.loads(i)
#   asp_dic=list(i.values())
#   for j in range(len(asp_list)):
#     asp_list[j].append(asp_dic[j])

# labels_dict={'ABS':sec_list[0],'INT':sec_list[1],'RWK':sec_list[2],'PDI':sec_list[3],'DAT':sec_list[4],'MET':sec_list[5],'EXP':sec_list[6],
#              'RES':sec_list[7],'TNF':sec_list[8],'ANA':sec_list[9],'FWK':sec_list[10],'OAL':sec_list[11],'BIB':sec_list[12],'EXT':sec_list[13],
#              'APR':asp_list[0],'NOV':asp_list[1],'IMP':asp_list[2],'CMP':asp_list[3],'PNF':asp_list[4],'REC':asp_list[5],'EMP':asp_list[6],'SUB':asp_list[7],'CLA':asp_list[8]}
# temp_df=pd.DataFrame(labels_dict)

# features_df=review_df[meta_feat+count_feat+hedgelist_feat+hedgepeer_feat+senti_feat+pos_feat+secasp_coverage_feat+secasp_dist_dic].copy()
# features_df=pd.concat([features_df, temp_df, review_df[scores+info_score].copy()], axis=1)
# features_df.to_csv('/content/drive/MyDrive/ML/innovation_lab/golden_info/Qual_features.csv', index=False)

# features_df


,review_id,review,sentence count,word count,avg. sentence length,avg. word length,vocab length,hedge words,non-hedge words,hedge ratio,...,CMP,PNF,REC,EMP,SUB,CLA,section score,aspect score,informativeness score,new info score
0,B1EA-M-0Z__R3,This paper presents a new covariance function ...,9,171,24.428571,5.165644,107,0,171,0.000000,...,0,0,0,3,0,1,0.571765,0.445688,0.328460,0.320953
1,ByuP8yZRb__R2,The below review addresses the first revision ...,20,402,17.478261,5.605820,191,3,399,0.007519,...,2,0,1,8,0,0,0.431786,0.337804,0.221011,0.215628
2,H1meywxRW__R1,The authors of this paper propose some extensi...,7,182,22.750000,5.398876,114,1,181,0.005525,...,1,1,1,2,0,1,0.429203,0.667289,0.306034,0.299479
3,NIPS_2018_184__R1,This paper presents a new type of neural netwo...,9,190,12.666667,5.227778,119,0,190,0.000000,...,0,0,0,2,0,0,0.358108,0.222844,0.164626,0.159561
4,NIPS_2018_184__R2,The paper presents SplineNets that reformulate...,11,263,16.437500,5.201646,152,2,261,0.007663,...,0,0,0,5,2,2,0.429849,0.558231,0.274240,0.271279
5,NIPS_2018_184__R3,"In this paper, the authors introduce SplineNet...",10,144,14.400000,5.650000,93,0,144,0.000000,...,1,1,0,4,1,0,0.715998,0.668533,0.513993,0.505257
6,NIPS_2018_443__R1,the paper introduces a differentiable clusteri...,6,251,25.100000,5.261411,150,1,250,0.004000,...,0,0,0,3,1,1,0.500907,0.445688,0.286606,0.282167
7,NIPS_2018_443__R2,"In this paper, the task of learning the hierar...",8,179,17.900000,5.682081,107,0,179,0.000000,...,0,0,0,1,1,1,0.572435,0.444492,0.328452,0.324882
8,NIPS_2018_443__R3,This paper presents a hierarchical graph repre...,9,195,21.666667,5.670213,111,1,194,0.005155,...,0,0,0,2,0,1,0.572061,0.445162,0.326769,0.323615
9,NIPS_2018_456__R1,- The problem is interesting and worth studyin...,3,22,7.333333,6.047619,18,0,22,0.000000,...,0,0,0,0,0,0,0.142976,0.000000,0.052598,0.050451
